In [2]:
%pip install numpy pandas pillow tensorflow opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp

  Using cached tensorflow-2.18.0-cp39-cp39-win_amd64.whl.metadata (3.3 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached tensorflow_intel-2.18.0-cp39-cp39-win_amd64.whl.metadata (4.9 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.12.23-py2.py3-none-any.whl.metadata (876 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.3-cp39-cp39-win_amd64.whl.metadata (592 bytes)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.1-cp39-cp39-win_amd64.whl.metadata (6.5 kB)
  Using cached grpcio-1.69.0-cp39-cp39-win_amd64.whl.metadata (

In [3]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
import cv2  
import matplotlib.pyplot as plt

print("ok")

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
import cv2  
import matplotlib.pyplot as plt
# Dataset paths
dataset_path = "/kaggle/input/moddelite/ModdeDataset"
jpeg_dataset_path = "/kaggle/working/MODDE_JPEG"

# Check if dataset path exists
if os.path.exists(dataset_path):
    print("Dataset exists.")
else:
    print("Paths aren't aligned properly. Please check the dataset path.")

# Function to convert images to JPEG format
def convert_images_to_jpeg(src_dir, dest_dir):
    # Create the destination directory if it doesn't exist
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    # Traverse through the source directory
    for root, _, files in os.walk(src_dir):
        for file in files:
            src_file_path = os.path.join(root, file)  # Source file path
            rel_path = os.path.relpath(root, src_dir)  # Relative path
            dest_dir_path = os.path.join(dest_dir, rel_path)  # Destination directory
            dest_file_path = os.path.join(dest_dir_path, file)  # Destination file path

            # Ensure the destination directory exists
            if not os.path.exists(dest_dir_path):
                os.makedirs(dest_dir_path)

            # Skip files that already exist
            if os.path.exists(dest_file_path):
                continue

            try:
                # Convert and save as JPEG
                with Image.open(src_file_path) as img:
                    if img.mode != 'RGB':  # Convert to RGB if needed
                        img = img.convert('RGB')
                    dest_file_path = os.path.splitext(dest_file_path)[0] + '.jpg'
                    img.save(dest_file_path, 'JPEG')
            except Exception as e:
                print(f"Failed to process {src_file_path}: {e}")

# Convert images to JPEG format
convert_images_to_jpeg(dataset_path, jpeg_dataset_path)

# Load the dataset
data = tf.keras.utils.image_dataset_from_directory(
    jpeg_dataset_path,
    labels='inferred',  # Automatically infer labels based on folder names
    label_mode='categorical',  # 'int', 'categorical', or None
    batch_size=32,  # Number of images per batch
    image_size=(256, 256),  # Resize images to a uniform size
    shuffle=True,  # Shuffle the dataset
    seed=123  # Seed for reproducibility
)

print(data.class_names)  # Check if the class names match your folder structure

# Optional: Print a summary of the dataset
for images, labels in data.take(1):
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(14, 10))  # Adjust the figure size for better spacing
shown_classes = set()  # Track displayed classes
grid_size = (3, 4)  # Grid size: Adjust according to the number of classes

# Iterate through the dataset to display one sample per class
for images, labels in data:
    for i in range(len(images)):
        label_index = np.argmax(labels[i].numpy())  # Get the class index
        if label_index not in shown_classes:
            ax = plt.subplot(grid_size[0], grid_size[1], len(shown_classes) + 1)  # Place in grid
            ax.imshow(images[i].numpy().astype(int))
            ax.set_title(
                data.class_names[label_index], 
                fontsize=12, 
                weight="bold", 
                color="#333333"  # Modern gray tone for title
            )
            ax.axis("off")  # Remove axes for a clean look
            shown_classes.add(label_index)
        if len(shown_classes) == len(data.class_names):  # Break if all classes are displayed
            break
    if len(shown_classes) == len(data.class_names):  # Break outer loop if done
        break

# Add a central title for the whole grid
plt.suptitle(
    "Sample Images from Each Class",
    fontsize=16,
    weight="bold",
    color="#222222",
    y=0.92
)

# Adjust layout for better spacing
plt.tight_layout(pad=2.0, rect=[0, 0, 1, 0.92])
plt.show()


In [ ]:
data=data.map(lambda x,y:(x/255,y)) #transformation for this pipline
data.as_numpy_iterator().next()[0].max()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

# Convolutional Layer and MaxPooling Layer
model.add(Conv2D(16, (3, 3), strides=1, activation='relu', input_shape=(256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), strides=1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3, 3), strides=1, activation='relu'))
model.add(MaxPooling2D())

# Flatten and Dense Layers
model.add(Flatten())
model.add(Dense(256, activation="relu"))

# Adjusted final Dense layer for multiclass classification
model.add(Dense(12, activation="softmax"))  # Replace 12 with the number of your classes

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Summary of the model
model.summary()


In [ ]:
logdir="logs"
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [ ]:
# Training the model
history = model.fit(
    train,  # Training dataset
    epochs=10,  # Number of epochs
    validation_data=val,  # Validation dataset
    callbacks=[tensorboard_callback]  # TensorBoard callback
)

# Calculate total accuracy and loss
mod_accuracy = history.history['accuracy'][-1]  # Accuracy from the last epoch
mod_loss = history.history['loss'][-1]  # Loss from the last epoch

print(f"Model Total Accuracy: {mod_accuracy * 100:.2f}%")
print(f"Model Total Loss: {mod_loss:.4f}")


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Ensure the history object is properly used
history_dict = history.history

# Set up the dark theme
plt.style.use("dark_background")
plt.figure(figsize=(10, 6))

# Use a modern font
font_path = fm.findfont(fm.FontProperties(family="Arial"))
font_properties = fm.FontProperties(fname=font_path)

# Plot the data with adjusted line width
plt.plot(history_dict["loss"], color="#1f77b4", linewidth=2, label="Training Loss")
plt.plot(history_dict["val_loss"], color="#ff7f0e", linewidth=2, label="Validation Loss")
plt.plot(history_dict["accuracy"], color="#d62728", linewidth=2, label="Training Accuracy")
plt.plot(history_dict["val_accuracy"], color="#2ca02c", linewidth=2, label="Validation Accuracy")

# Add title and labels
plt.title("Model Training History", fontsize=16, fontproperties=font_properties, pad=20)
plt.xlabel("Epochs", fontsize=14, fontproperties=font_properties, labelpad=10)
plt.ylabel("Metric Values", fontsize=14, fontproperties=font_properties, labelpad=10)

# Add legend outside the plot
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.1), ncol=2, fontsize=12, frameon=False)

# Add total accuracy and loss as annotations
mod_accuracy = history_dict['accuracy'][-1]  # Accuracy from the last epoch
mod_loss = history_dict['loss'][-1]  # Loss from the last epoch
text = f"Model Total Accuracy: {mod_accuracy * 100:.2f}%\nModel Total Loss: {mod_loss:.4f}"
plt.gcf().text(0.95, 0.5, text, fontsize=12, fontproperties=font_properties, color="white", ha="right", va="center")

# Adjust layout
plt.tight_layout(rect=[0, 0, 0.9, 1])

# Show the plot
plt.show()


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

# Load and preprocess the test image
img = cv2.imread("/kaggle/input/moddelite/ModdeDataset/skirt/skirt (1012).jpg")

# Check if the image loaded correctly
if img is None:
    raise ValueError("Error: Image not loaded. Check the file path.")

# Convert BGR to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize image to model's expected input size (512, 512)
resized_img = cv2.resize(img, (256, 256))

# Show the image
plt.imshow(resized_img)
plt.axis('off')
plt.show()

# Normalize image and expand dimensions to match model input
input_image = np.expand_dims(resized_img, axis=0).astype(np.float32) / 255.0

# Predict the class probabilities
yhat = model.predict(input_image)

# Get the class index with the highest probability
predicted_class = np.argmax(yhat, axis=1)[0]
print(f'Predicted Class: {predicted_class}')

data = tf.keras.utils.image_dataset_from_directory(
    jpeg_dataset_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=123
)

class_names = data.class_names  # Extract class names
# Print the predicted class label
print(f'The image you provided is a: {class_names[predicted_class]}')

